In [1]:
from pathlib import Path

In [2]:
import sys

sys.path.append(str(Path.home() / Path("Development") / "iSparrow"))

In [3]:
from iSparrow import PreprocessorBase
from iSparrow import ModelBase
from iSparrow import SparrowRecording
from iSparrow import SpeciesPredictorBase
from iSparrow import SparrowWatcher
import iSparrow.utils as utils

2024-04-10 11:30:34.336737: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-10 11:30:34.831349: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import os
import pandas as pd
import tests.set_up_sparrow_env as sp
from watchdog.events import LoggingEventHandler, FileSystemEventHandler
from watchdog.observers import Observer
import time
from datetime import datetime
from copy import deepcopy
import yaml
import threading


In [5]:
# make a mock install of sparrow. will be invisible in the future
sp.install()

Creating iSparrow folders and downloading data... 
...reading config from /home/hmack/Development/iSparrow/config/install_cfg.yml
...Making direcotries...
/home/hmack/iSparrow /home/hmack/iSparrow/models /home/hmack/iSparrow_data /home/hmack/iSparrow_output /home/hmack/iSparrow/example
... Downloading model files...
... Downloading example files...
Installation finished


In [6]:
preprocessor_cfg = {
    "sample_rate": 48000,
    "overlap": 0.0,
    "sample_secs": 3.0,
    "resample_type": "kaiser_fast",
}

model_cfg = {
    "num_threads": 1,
    "sigmoid_sensitivity": 1.0,
    "species_list_file": None,
}

recording_cfg = {
    "date": datetime(year=2022, month=5, day=10),
    "lat": 35.4244,
    "lon": -120.7463,
    "species_presence_threshold": 0.03,
    "min_conf": 0.25,
    "species_predictor": None,
}

species_predictor_cfg = {
    "use_cache": True,
    "num_threads": 1,
}

runner = SparrowWatcher(
    Path.home() / "iSparrow_data",
    Path.home() / "iSparrow_output",
    Path.home() / "iSparrow/models",
    "birdnet_default",
    preprocessor_config=preprocessor_cfg,
    model_config=model_cfg,
    recording_config=recording_cfg,
    species_predictor_config=species_predictor_cfg,
)

Labels loaded.


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
runner.start()

read audio data of duration 120.0 from /home/hmack/iSparrow_data/Q9_0#.wav with sampling rate 48000
process audio data default
process audio data default: complete, read  40 chunks.
name of model:  birdnet_default


[{'start': 9.0, 'end': 12.0, 'label': 'Haemorhous mexicanus_House Finch', 'confidence': 0.63937765}, {'start': 33.0, 'end': 36.0, 'label': 'Junco hyemalis_Dark-eyed Junco', 'confidence': 0.45906296}, {'start': 36.0, 'end': 39.0, 'label': 'Junco hyemalis_Dark-eyed Junco', 'confidence': 0.353685}, {'start': 39.0, 'end': 42.0, 'label': 'Haemorhous mexicanus_House Finch', 'confidence': 0.25325066}, {'start': 42.0, 'end': 45.0, 'label': 'Junco hyemalis_Dark-eyed Junco', 'confidence': 0.7375431}, {'start': 51.0, 'end': 54.0, 'label': 'Haemorhous mexicanus_House Finch', 'confidence': 0.25640452}, {'start': 54.0, 'end': 57.0, 'label': 'Haemorhous mexicanus_House Finch', 'confidence': 0.6070655}, {'start': 60.0, 'end': 63.0, 'label': 'Junco hyemalis_Dark-eyed Junco', 'confidence': 0.554988}, {'start': 69.0, 'end': 72.0, 'label': 'Haemorhous mexicanus_House Finch', 'confidence': 0.29147243}, {'start': 72.0, 'end': 75.0, 'label': 'Haemorhous mexicanus_House Finch', 'confidence': 0.56384665}, {'st

In [ ]:
preprocessor_cfg = {
    "sample_rate": 48000,
    "overlap": 0.0,
    "sample_secs": 3.0,
    "resample_type": "kaiser_fast",
}

model_cfg = {
    "num_threads": 1,
    "sigmoid_sensitivity": 1.0,
    "default_model_path": str(Path.home() / "iSparrow/models/birdnet_default"),
}

runner.change_analyzer(
    "birdnet_custom", preprocessor_config=preprocessor_cfg, model_config=model_cfg
)